### This notebook will be used for the Coursera Capstone Project to cluster the neighborhoods in Toronto


In [0]:
# Importing all the libraries
import requests
# !pip install beautifulsoup4
from bs4 import BeautifulSoup
import pandas as pd

2) Scraping table from following Wikipedia page using BeautifulSoup and transforming the data into dataframe:

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [182]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
list = []
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
table = soup.find('table',attrs={'class':'wikitable'})
table_body = table.tbody
rows = table_body.find_all('tr')
for row in rows:
  cols = row.find_all('td')
  if(len(cols)!=0):
    cells = [col.text.strip() for col in cols]
    list.append([ele for ele in cells])
df = pd.DataFrame(list, columns=columns)
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ignoring cells with a borough that is Not assigned

In [183]:
df = df[df['Borough']!='Not assigned']
df.reset_index(drop=True,inplace=True)
df

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


Merging Two or more neighborhood with same postcode into one column

In [184]:
df=df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Assigning neighborhood same as the borough where cell is having borough but Not assigned neighborhood.

In [0]:
df.Neighborhood[df.Neighborhood=='Not assigned']=df.Borough[df.Neighborhood=='Not assigned']


Checking no of rows in the data frame

In [186]:
print("No of Rows are {}".format(df.shape[0]))

No of Rows are 103


Reading PostCodes from Geospatial_Coordinates.csv

In [187]:
postalCodeDF = pd.read_csv("https://raw.githubusercontent.com/arora-raghav/Coursera_Capstone/master/Geospatial_Coordinates.csv")
postalCodeDF.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging Two Dataframes to get the required columns

In [191]:
postalCodeDF = postalCodeDF.rename(columns = {'Postal Code':'Postcode'})
postalCodeDF
df = df.merge(postalCodeDF, how='left',on='Postcode')
df

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
